In [1]:
import math

class Location:
    def __init__(self, lat, lon):
        self.__lat = lat
        self.__lon = lon
    def get_lat(self):
        return self.__lat
    def get_lon(self):
        return self.__lon
    def set_precision(self, precision):
        lt = round(self.__lat, precision)
        ln = round(self.__lon, precision)
        loc = Location(lt, ln)
        return loc
    def __degToRad(self, deg):
        return deg * math.pi / 180
    def distance(self, point):
        R = 6371e3
        lat1 = self.__degToRad(self.__lat)
        lat2 = self.__degToRad(point.get_lat())
        deltaLat = self.__degToRad(point.get_lat() - self.__lat)
        deltaLon = self.__degToRad(point.get_lon() - self.__lon)
        a = math.sin(deltaLat / 2) * math.sin(deltaLat / 2) + math.cos(lat1) * math.cos(lat2) * math.sin(deltaLon / 2) * math.sin(deltaLon / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

In [2]:
class Region:
    def __init__(self, lat_range, lon_range):
        self.__lat_range = lat_range
        self.__lon_range = lon_range
        self.__hotspots = []
        self.__frequency = dict()
    def get_range(self):
        return self.__lat_range, self.__lon_range
    def get_neighbours(self):
        pass
    def get_hotspots(self):
        return self.__hotspots
    def __sort_hotspots(self):
        self.__hotspots.sort(key=lambda x: self.__frequency[x], reverse=True)
    def add_spot(self, spot, freq):
        spot = spot.set_precision(4)
        spot = (spot.get_lat(), spot.get_lon())
        idx = self.__hotspots.index(spot) if spot in self.__hotspots else -1
        # print(idx)
        if idx == -1:
            self.__hotspots.append(spot)
            self.__frequency[spot] = freq
        else:
            self.__frequency[spot] += freq
        self.__sort_hotspots()


In [8]:
degreeLat = 7
degreeLon = 9
lat1000 = 0.009
lon1000 = 0.0147
# lat1000 = 1
# lon1000 = 1
lat_cells = math.ceil(degreeLat / lat1000)
lon_cells = math.ceil(degreeLon / lon1000)
lat_mid = lat_cells // 2
lon_mid = lon_cells // 2

print("Lat Cells:", lat_cells) 
print("Lon Cells:", lon_cells)
print("Total Regions:", lat_cells * lon_cells)
print("Max Locations in a Region:", math.ceil((lat1000 / .0001) * (lon1000 / .0001)))

Lat Cells: 778
Lon Cells: 613
Total Regions: 476914
Max Locations in a Region: 13230


In [116]:
from IPython.display import display, clear_output

"""
Generated For Jammu & Kashmir Region
Latitude Range - [30, 37]
Longitude Range - [71, 80]
"""

world_map = []
lat = 30
it = 0
print()
for i in range(lat_cells):
    lon = 71
    world_map.append([])
    for j in range(lon_cells):
        world_map[i].append(Region((lat, lat + lat1000), (lon, lon + lon1000)))
        lon += lon1000
    if it % 10 == 0 or it == lat_cells - 1:
        clear_output(wait=True)
        display(f'{it + 1} / {lat_cells}, {round(((it + 1) * 100) / lat_cells, 2)}%')
    it += 1
    lat += lat1000

'778 / 778, 100.0%'

In [10]:
def location_to_index(loc):
    lat = round(loc.get_lat(), 4)
    lon = round(loc.get_lon(), 4)
    lat_idx = math.floor((lat - 30) // lat1000)
    lon_idx = math.floor((lon - 71) // lon1000)
    return (lat_idx, lon_idx)

In [11]:
print(location_to_index(Location(37, 80)))
print(location_to_index(Location(30, 71)))

(777, 612)
(0, 0)


In [16]:
def get_obfuscated_loc(loc, range):
    idx = location_to_index(loc)
    r = world_map[idx[0]][idx[1]]
    hotspots = r.get_hotspots()
    for spot in hotspots:
        if range[0] <= loc.distance(Location(spot[0], spot[1])) <= range[1]:
            return Location(spot[0], spot[1])

In [81]:
import random

def generate_random_hotspots(world_map, cell_index, count):
    region = world_map[cell_index[0]][cell_index[1]]
    lat_range, lon_range = region.get_range()
    hotspots = []
    frequencies = []
    for i in range(count):
        lat = round(random.random() * (lat_range[1] - lat_range[0]) + lat_range[0], 4)
        lon = round(random.random() * (lon_range[1] - lon_range[0]) + lon_range[0], 4)
        hotspots.append(Location(lat, lon))
        frequencies.append(random.randint(0, 2))
    return hotspots, frequencies

In [82]:
def add_hotspots_to_cell(world_map, cell_index, hotspots, frequencies):
    region = world_map[cell_index[0]][cell_index[1]]
    for spot, freq in zip(hotspots, frequencies):
        region.add_spot(spot, freq)

In [117]:
user_lat = 32.002
user_lon = 77.005
user_loc = Location(user_lat, user_lon)

cell_index = location_to_index(user_loc)
hotspots, frequencies = generate_random_hotspots(world_map, cell_index, 100)
add_hotspots_to_cell(world_map, cell_index, hotspots, frequencies)

In [120]:
obs_loc = get_obfuscated_loc(user_loc, [500, 700])
print(obs_loc.get_lat(), obs_loc.get_lon())

31.9991 76.9997


In [53]:
# import folium
# import webbrowser

# lats = [user_lat, obs_loc.get_lat()]
# lons = [user_lon, obs_loc.get_lon()]
# map = folium.Map(location=[sum(lats) / len(lats), sum(lons) / len(lons)], zoom_start=20)

# folium.Marker([obs_loc.get_lat(), obs_loc.get_lon()]).add_to(map)
# folium.Marker([user_lat, user_lon], icon=folium.Icon(color='black',icon_color='#FFFF00')).add_to(map)

# map.save('obs.html')
# webbrowser.open('obs.html')

True

In [121]:
import folium
import webbrowser

lats = []
lons = []
map = folium.Map(location=[user_lat, user_lon], zoom_start=16)

colors = ['red', 'green', 'blue']

for loc, freq in zip(hotspots, frequencies):
    lat, lon = loc.get_lat(), loc.get_lon()
    folium.Marker([lat, lon], icon=folium.Icon(color=colors[freq])).add_to(map)
folium.Marker([user_lat, user_lon], icon=folium.Icon(color='black',icon_color='#FFFF00')).add_to(map)
# folium.Marker([obs_loc.get_lat(), obs_loc.get_lon()], icon=folium.Icon(color='black',icon_color='#FFFF00')).add_to(map)

line_coords = [[obs_loc.get_lat(), obs_loc.get_lon()], [user_lat, user_lon]]
fg = folium.FeatureGroup("Lines")
folium.PolyLine(line_coords).add_to(fg)
fg.add_to(map)
folium.LayerControl(position='bottomright').add_to(map)

map.save('obs.html')
webbrowser.open('obs.html')

True

In [26]:
pos = list()
pos.append(Location(32.758, 74.89639))     # dist = 0
pos.append(Location(32.758469, 74.896626)) # dist = 50+
pos.append(Location(32.757593, 74.895512)) # dist = 90+
pos.append(Location(32.759156, 74.895684)) # dist = 140+
pos.append(Location(32.762922, 74.895594)) # dist = 500+
pos.append(Location(32.764816, 74.897932)) # dist = 700+
print(pos[0].distance(pos[5]))

771.4983233777218


In [28]:
idx = location_to_index(pos[3])
print("Index:", idx)
r = world_map[idx[0]][idx[1]]
print("Lat-Lon Range:", r.get_range())

Index: (306, 265)
Lat-Lon Range: ((32.754000000000104, 32.763000000000105), (74.89550000000128, 74.91020000000128))


In [29]:
for p in pos[1: ]:
    idx = location_to_index(p)
    r = world_map[idx[0]][idx[1]]
    r.add_spot(p)

In [30]:
ob_l = get_obfuscated_loc(pos[0], (50, 100))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (100, 150))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (150, 500))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()
ob_l = get_obfuscated_loc(pos[0], (500, 1000))
if ob_l: 
    print(ob_l.get_lat(), ob_l.get_lon())
    print(pos[0].distance(ob_l))
    print()

32.7585 74.8966
58.963536580550524

32.7592 74.8957
148.21506648370803

32.7629 74.8956
549.8400958017124

